In [142]:
import numpy as np
import pandas as pd

import glob
import re
import math

# user='jeremyong'
txt='behminyan'

In [143]:
users=[]
folders=glob.glob("data/train/*") #PLS EDIT

for fd in folders:
    users.append(fd[fd.rfind('/')+1:])
    
print 'USERNAMES:', users

USERNAMES: ['behminyan', 'changsheng', 'jeremyong']


In [144]:
def calcbearing(pointA, pointB):
    """
    Calculates the bearing between two points.
    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `pointA: The tuple representing the latitude/longitude for the
        first point. Latitude and longitude must be in decimal degrees
      - `pointB: The tuple representing the latitude/longitude for the
        second point. Latitude and longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    
    Source - https://gist.github.com/jeromer/2005586
    """
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

In [145]:
df_rows = []
cols=["text","user"]
letters=""

for user in users:
    filenames=glob.glob("data/train/"+user+"/"+txt+"*.csv")
    print 'FILENAMES:', filenames
    
    for filename in filenames:

        input = pd.read_csv(filename)
        userrow={}        
        userrow['text'] = txt
        userrow['user'] = user
        
        letters=input.letter
        xdown=input.down_x
        ydown=input.down_y
        xup=input.up_x
        yup=input.up_y
        
        for i in range(len(letters)):
            if (letters[i]=='return'):
                break
            userrow[letters[i]+str(i)+'_xdown']=xdown[i]
            userrow[letters[i]+str(i)+'_ydown']=ydown[i]
            userrow[letters[i]+str(i)+'_xup']=xup[i]
            userrow[letters[i]+str(i)+'_yup']=yup[i]
            userrow[letters[i]+str(i)+'_b']=calcbearing((xdown[i],ydown[i]),(xup[i],yup[i]))

        df_rows.append(userrow)
    
for i in range(len(letters)): # SOON TO CHANGE
    if (letters[i]=='return'):
        break
    cols.append(letters[i]+str(i)+'_xdown')
    cols.append(letters[i]+str(i)+'_ydown')
    cols.append(letters[i]+str(i)+'_xup')
    cols.append(letters[i]+str(i)+'_yup')
    cols.append(letters[i]+str(i)+'_b')

FILENAMES: ['data/train/behminyan/behminyan_1.csv', 'data/train/behminyan/behminyan_2.csv', 'data/train/behminyan/behminyan_3.csv', 'data/train/behminyan/behminyan_4.csv', 'data/train/behminyan/behminyan_5.csv']
FILENAMES: ['data/train/changsheng/behminyan_1.csv', 'data/train/changsheng/behminyan_10.csv', 'data/train/changsheng/behminyan_2.csv', 'data/train/changsheng/behminyan_3.csv', 'data/train/changsheng/behminyan_4.csv', 'data/train/changsheng/behminyan_5.csv', 'data/train/changsheng/behminyan_6.csv', 'data/train/changsheng/behminyan_7.csv', 'data/train/changsheng/behminyan_8.csv', 'data/train/changsheng/behminyan_9.csv']
FILENAMES: ['data/train/jeremyong/behminyan_1.csv', 'data/train/jeremyong/behminyan_2.csv', 'data/train/jeremyong/behminyan_3.csv', 'data/train/jeremyong/behminyan_4.csv', 'data/train/jeremyong/behminyan_5.csv']


In [146]:
combined = pd.DataFrame(df_rows,columns=cols)

In [147]:
# Taken from: https://www.peterbe.com/plog/uniqifiers-benchmark
def unique (seq, idfun=None): # order preserving
    if idfun is None:
        def idfun(x): 
            return x
    seen = {}
    result = []
    for item in seq:
        marker = idfun(item)
        if marker in seen: 
            continue
        seen[marker] = 1
        result.append(item)
    return result

In [148]:
cats=combined['user'].tolist()
catlist=unique(cats)
print catlist

combined['cat'] = [catlist.index(x) for x in combined.user]
combined.head()

['behminyan', 'changsheng', 'jeremyong']


,text,user,b0_xdown,b0_ydown,b0_xup,b0_yup,b0_b,e1_xdown,e1_ydown,e1_xup,...,a9_ydown,a9_xup,a9_yup,a9_b,n10_xdown,n10_ydown,n10_xup,n10_yup,n10_b,cat
0,behminyan,behminyan,61.020408,63.079777,61.020408,63.079777,0,26.632653,14.656772,26.632653,...,37.291280,8.163265,37.291280,0,66.836735,56.771800,66.836735,56.771800,0,0
1,behminyan,behminyan,61.020408,63.079777,61.020408,63.079777,0,26.632653,14.656772,26.632653,...,37.291280,8.163265,37.291280,0,66.836735,56.771800,66.836735,56.771800,0,0
2,behminyan,behminyan,61.938776,66.233766,61.938776,66.233766,0,27.551020,7.792208,27.551020,...,34.693878,9.897959,34.693878,0,67.551020,57.328386,67.551020,57.328386,0,0
3,behminyan,behminyan,59.897959,61.410019,59.897959,61.410019,0,27.653061,10.018553,27.653061,...,29.128015,8.877551,29.128015,0,65.918367,57.699443,65.918367,57.699443,0,0
4,behminyan,behminyan,61.938776,63.636364,61.938776,63.636364,0,26.326531,14.842301,26.326531,...,33.951763,10.102041,33.951763,0,68.163265,63.079777,68.163265,63.079777,0,0


# Recursive Feature Elimination

In [149]:
traindata=combined.drop(['text','user','cat'], axis=1).as_matrix(columns=None)
labels=combined['cat'].tolist()

In [150]:
# Recursive Feature Elimination
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# create a base classifier used to evaluate a subset of attributes
model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model, 3)
# rfe = rfe.fit(dataset.data, dataset.target)
rfe = rfe.fit(traindata, labels)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[False False False False False False False False False False False False
 False False False False  True False False False False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False  True False]
[38 43 37 44 51 15 33 11 31 47 16 13 12  8 46  9  1  3  5 42 36 17 40 23 45
 32 18 35  1 24 20  6 22 19 52  7 21  2 26 50 25 30 27 34 48 28 10 29  4 49
 39 14 41  1 53]


In [151]:
train_features=cols[2:]
print train_features


['b0_xdown', 'b0_ydown', 'b0_xup', 'b0_yup', 'b0_b', 'e1_xdown', 'e1_ydown', 'e1_xup', 'e1_yup', 'e1_b', 'h2_xdown', 'h2_ydown', 'h2_xup', 'h2_yup', 'h2_b', 'space3_xdown', 'space3_ydown', 'space3_xup', 'space3_yup', 'space3_b', 'm4_xdown', 'm4_ydown', 'm4_xup', 'm4_yup', 'm4_b', 'i5_xdown', 'i5_ydown', 'i5_xup', 'i5_yup', 'i5_b', 'n6_xdown', 'n6_ydown', 'n6_xup', 'n6_yup', 'n6_b', 'space7_xdown', 'space7_ydown', 'space7_xup', 'space7_yup', 'space7_b', 'y8_xdown', 'y8_ydown', 'y8_xup', 'y8_yup', 'y8_b', 'a9_xdown', 'a9_ydown', 'a9_xup', 'a9_yup', 'a9_b', 'n10_xdown', 'n10_ydown', 'n10_xup', 'n10_yup', 'n10_b']


In [152]:
shortlist1=pd.DataFrame({'feature': train_features, 'importance':rfe.ranking_}).sort_values(by=['importance'],ascending=True)
shortlist1.head(10)

,feature,importance
28,i5_yup,1
16,space3_ydown,1
53,n10_yup,1
37,space7_xup,2
17,space3_xup,3
48,a9_yup,4
18,space3_yup,5
31,n6_ydown,6
35,space7_xdown,7
13,h2_yup,8


In [153]:
shortlist1=shortlist1.head(10)['feature'].tolist()
print shortlist1

['i5_yup', 'space3_ydown', 'n10_yup', 'space7_xup', 'space3_xup', 'a9_yup', 'space3_yup', 'n6_ydown', 'space7_xdown', 'h2_yup']


# Ensemble Feature Importance

In [154]:
# Feature Importance
# from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

# fit an Extra Trees model to the data
model = RandomForestClassifier()
model.fit(traindata, labels)
# display the relative importance of each attribute
print(model.feature_importances_)

[ 0.          0.          0.          0.          0.          0.01661475
  0.          0.05547074  0.          0.          0.          0.          0.
  0.04422572  0.          0.10521781  0.          0.06557228  0.10124715
  0.01466276  0.13321914  0.          0.07213266  0.          0.          0.
  0.          0.          0.01335878  0.          0.          0.          0.
  0.0526699   0.          0.1         0.          0.          0.13075106
  0.          0.          0.01377953  0.0279965   0.          0.          0.
  0.          0.          0.05308123  0.          0.          0.          0.
  0.          0.        ]


In [155]:
shortlist2=pd.DataFrame({'feature':train_features, 'importance':model.feature_importances_}).sort_values(by=['importance'],ascending=False)
shortlist2.head(10)

,feature,importance
20,m4_xdown,0.133219
38,space7_yup,0.130751
15,space3_xdown,0.105218
18,space3_yup,0.101247
35,space7_xdown,0.100000
22,m4_xup,0.072133
17,space3_xup,0.065572
7,e1_xup,0.055471
48,a9_yup,0.053081
33,n6_yup,0.052670


In [156]:
shortlist2=shortlist2.head(10)['feature'].tolist()
print shortlist2

['m4_xdown', 'space7_yup', 'space3_xdown', 'space3_yup', 'space7_xdown', 'm4_xup', 'space3_xup', 'e1_xup', 'a9_yup', 'n6_yup']


# Merge shortlists

In [162]:
# print cols
shortlist=unique(shortlist1 + shortlist2)
print shortlist

['i5_yup', 'space3_ydown', 'n10_yup', 'space7_xup', 'space3_xup', 'a9_yup', 'space3_yup', 'n6_ydown', 'space7_xdown', 'h2_yup', 'm4_xdown', 'space7_yup', 'space3_xdown', 'm4_xup', 'e1_xup', 'n6_yup']


# New input data

In [163]:
df_rows = []
cols=["text","user"]
letters=""

for user in users:
    filenames=glob.glob("data/test/"+user+"/"+txt+"*.csv")
    print 'FILENAMES:', filenames
    
    for filename in filenames:

        input = pd.read_csv(filename)
        userrow={}        
        userrow['text'] = txt
        userrow['user'] = user
        
        letters=input.letter
        xdown=input.down_x
        ydown=input.down_y
        xup=input.up_x
        yup=input.up_y
        
        for i in range(len(letters)):
            if (letters[i]=='return'):
                break
            userrow[letters[i]+str(i)+'_xdown']=xdown[i]
            userrow[letters[i]+str(i)+'_ydown']=ydown[i]
            userrow[letters[i]+str(i)+'_xup']=xup[i]
            userrow[letters[i]+str(i)+'_yup']=yup[i]
            userrow[letters[i]+str(i)+'_b']=calcbearing((xdown[i],ydown[i]),(xup[i],yup[i]))

        df_rows.append(userrow)
    
for i in range(len(letters)): # SOON TO CHANGE
    if (letters[i]=='return'):
        break
    cols.append(letters[i]+str(i)+'_xdown')
    cols.append(letters[i]+str(i)+'_ydown')
    cols.append(letters[i]+str(i)+'_xup')
    cols.append(letters[i]+str(i)+'_yup')
    cols.append(letters[i]+str(i)+'_b')

FILENAMES: []
FILENAMES: ['data/test/changsheng/behminyan_1.csv', 'data/test/changsheng/behminyan_10.csv', 'data/test/changsheng/behminyan_11.csv', 'data/test/changsheng/behminyan_12.csv', 'data/test/changsheng/behminyan_13.csv', 'data/test/changsheng/behminyan_14.csv', 'data/test/changsheng/behminyan_15.csv', 'data/test/changsheng/behminyan_16.csv', 'data/test/changsheng/behminyan_17.csv', 'data/test/changsheng/behminyan_18.csv', 'data/test/changsheng/behminyan_19.csv', 'data/test/changsheng/behminyan_2.csv', 'data/test/changsheng/behminyan_20.csv', 'data/test/changsheng/behminyan_21.csv', 'data/test/changsheng/behminyan_22.csv', 'data/test/changsheng/behminyan_23.csv', 'data/test/changsheng/behminyan_24.csv', 'data/test/changsheng/behminyan_25.csv', 'data/test/changsheng/behminyan_26.csv', 'data/test/changsheng/behminyan_27.csv', 'data/test/changsheng/behminyan_28.csv', 'data/test/changsheng/behminyan_29.csv', 'data/test/changsheng/behminyan_3.csv', 'data/test/changsheng/behminyan_30

In [164]:
newdata = pd.DataFrame(df_rows,columns=cols)
newdata.head()

,text,user,b0_xdown,b0_ydown,b0_xup,b0_yup,b0_b,e1_xdown,e1_ydown,e1_xup,...,a9_xdown,a9_ydown,a9_xup,a9_yup,a9_b,n10_xdown,n10_ydown,n10_xup,n10_yup,n10_b
0,behminyan,changsheng,63.200000,66.504854,63.200000,66.504854,0,25.333333,20.873786,25.333333,...,8.800000,36.893204,8.800000,36.893204,0,68.8,63.592233,68.8,63.592233,0
1,behminyan,changsheng,61.066667,64.077670,61.066667,64.077670,0,23.200000,19.902913,23.200000,...,10.133333,42.718447,10.133333,42.718447,0,68.8,64.563107,68.8,64.563107,0
2,behminyan,changsheng,60.800000,66.504854,60.800000,66.504854,0,25.333333,12.621359,25.333333,...,13.066667,40.291262,13.066667,40.291262,0,68.8,61.650485,68.8,61.650485,0
3,behminyan,changsheng,61.866667,70.388350,61.866667,70.388350,0,26.666667,22.815534,26.666667,...,11.466667,39.320388,11.466667,39.320388,0,68.8,61.650485,68.8,61.650485,0
4,behminyan,changsheng,60.533333,64.077670,60.533333,64.077670,0,25.866667,18.446602,25.866667,...,12.533333,41.262136,12.533333,41.262136,0,65.6,61.165049,65.6,61.165049,0


# Make Summary

In [165]:
shortlist=['i5_yup', 'space3_ydown', 'n10_yup', 'space7_xup', 'space3_xup', 'a9_yup', 'space3_yup', 'n6_ydown', 'space7_xdown', 'h2_yup', 'm4_xdown', 'space7_yup', 'space3_xdown', 'm4_xup', 'e1_xup', 'n6_yup']
shortlist.append('user')
dataset=combined[shortlist].drop('user', axis=1).as_matrix(columns=None)
labels=combined['user']

In [166]:
test=newdata[shortlist].drop('user', axis=1).as_matrix(columns=None)
newdata[shortlist].head()

,i5_yup,space3_ydown,n10_yup,space7_xup,space3_xup,a9_yup,space3_yup,n6_ydown,space7_xdown,h2_yup,m4_xdown,space7_yup,space3_xdown,m4_xup,e1_xup,n6_yup,user
0,13.592233,82.038835,63.592233,68.800000,62.666667,36.893204,82.038835,62.135922,68.800000,46.116505,80.800000,81.553398,62.666667,80.800000,25.333333,62.135922,changsheng
1,21.359223,83.009709,64.563107,69.066667,60.000000,42.718447,83.009709,61.650485,69.066667,49.029126,81.066667,80.582524,60.000000,81.066667,23.200000,61.650485,changsheng
2,17.475728,78.155340,61.650485,68.533333,57.600000,40.291262,78.155340,63.592233,68.533333,43.689320,81.600000,80.097087,57.600000,81.600000,25.333333,63.592233,changsheng
3,18.932039,82.524272,61.650485,66.133333,58.400000,39.320388,82.524272,61.650485,67.200000,45.145631,78.666667,79.126214,58.400000,78.666667,26.666667,61.650485,changsheng
4,21.359223,83.009709,61.165049,69.066667,57.600000,41.262136,83.009709,62.621359,69.066667,41.262136,79.466667,81.067961,57.600000,79.200000,25.866667,62.621359,changsheng


In [167]:
def separateByClass(dataset,labels):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i] # vector = row of data
        label = labels[i]
        if (label not in separated):
            separated[label] = []
        separated[label].append(vector)
    return separated

In [168]:
sep=separateByClass(dataset,labels)
# print sep

In [169]:
import math
def mean(numbers):
    return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

# numbers = [1,2,3,4,5]
# print('Summary of {0}: mean={1}, stdev={2}').format(numbers, mean(numbers), stdev(numbers))

In [170]:
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    return summaries

In [171]:
summary=summarize(dataset)
print('Attribute summaries: {0}').format(summary)

Attribute summaries: [(14.392303258463002, 5.079312336251641), (85.20678350775502, 3.494018072945294), (63.597546697390001, 3.880517745678771), (61.959387755094987, 10.35866616803766), (60.049387755105023, 7.190552669961612), (39.188401750799997, 4.561991023722723), (85.212502476730009, 3.464114566116696), (62.040230920254999, 6.252954093843746), (61.959387755094987, 10.35866616803766), (37.583532971874995, 5.880788261046071), (80.291768707484991, 2.019277747861579), (85.004638218930012, 4.244402745894059), (59.927755102045012, 7.078652399040225), (80.181972789115008, 1.8599338952480577), (24.227006802724997, 2.1590733839055845), (62.040230920254999, 6.252954093843746)]


In [172]:
def summarizeByClass(dataset,labels):
    separated = separateByClass(dataset,labels)
    summaries = {}
    for classValue, instances in separated.iteritems():
        summaries[classValue] = summarize(instances)
    return summaries

In [173]:
summaries = summarizeByClass(dataset,labels)
print('Summary by class value: {0}').format(summaries)

Summary by class value: {'behminyan': [(8.4230055658519998, 2.1380126591169866), (85.306122448980005, 0.6346059879795103), (58.330241187360002, 2.684082428791891), (66.204081632640012, 1.104683151237325), (67.224489795920007, 1.4846851111150423), (34.471243042659999, 3.3456962015037104), (85.231910946219998, 0.6097097115011855), (54.656771799599994, 1.9262853213139188), (66.204081632640012, 1.104683151237325), (30.204081632640005, 2.558683043875136), (79.367346938780003, 2.060365551481327), (79.777365491659992, 1.6749044606141301), (66.897959183680001, 1.1422192097040336), (79.367346938780003, 2.060365551481327), (26.95918367346, 0.6010888357456833), (54.656771799599994, 1.9262853213139188)], 'changsheng': [(17.912621359220001, 4.090043895274053), (82.621359223310009, 1.8421034913794512), (65.339805825219997, 1.9711899459830737), (68.47999999999999, 1.1845924443703244), (62.160000000009994, 1.309113355589619), (42.087378640759994, 2.195525164171797), (82.669902912639998, 1.802591901840

In [174]:
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [ ]:
# x = 71.5
# mean = 73
# stdev = 6.2
# probability = calculateProbability(x, mean, stdev)
# print('Probability of belonging to this class: {0}').format(probability)

In [175]:
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.iteritems():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            
            prob = calculateProbability(x, mean, stdev)
            
            if(prob != 0):
                probabilities[classValue] *= math.log(prob)
            
            probabilities[classValue]=round(probabilities[classValue],2)
    return probabilities

In [176]:
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.iteritems():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [177]:
total=0
correct=0
for i in range(len(test)):
    total+=1
    testcase = test[i]
    result = predict(summaries, testcase)
    if(result=='changsheng'):
        correct+=1
    print(' {0} >>> {1}').format(result, calculateClassProbabilities(summaries, testcase))
print ('Accuracy >>> {0}').format(round(correct*100.0/total,4))

 jeremyong >>> {'behminyan': 281843171809.63, 'changsheng': 77602.88, 'jeremyong': 710284341183110.0}
 jeremyong >>> {'behminyan': 26800894313216.47, 'changsheng': 66622.42, 'jeremyong': 9109624775350936.0}
 jeremyong >>> {'behminyan': 252893502442661.94, 'changsheng': 12010454.58, 'jeremyong': 3774781258950110.5}
 jeremyong >>> {'behminyan': 27870263781.59, 'changsheng': 55159041.19, 'jeremyong': 91824146453069.81}
 jeremyong >>> {'behminyan': 1652028084653.6, 'changsheng': 5021732.41, 'jeremyong': 91185035330327.23}
 jeremyong >>> {'behminyan': 22072262910565.58, 'changsheng': 55079213.82, 'jeremyong': 1.439846379566133e+16}
 jeremyong >>> {'behminyan': 303849032767.49, 'changsheng': 3130525.88, 'jeremyong': 152246521274509.47}
 jeremyong >>> {'behminyan': 6717728913948.9, 'changsheng': 167907392.78, 'jeremyong': 377873059271003.4}
 behminyan >>> {'behminyan': 14015415975630.91, 'changsheng': 58190341.77, 'jeremyong': 1418629721876.52}
 jeremyong >>> {'behminyan': 2185725838699.3, 'c